In [1]:
import pandas as pd 
import matplotlib.pyplot as plt 
import numpy as np
import seaborn as sns
import scipy.stats as stats
import statsmodels.stats.api as sms

from IPython.display import display
from IPython.display import Markdown as md
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

def f():
    pd.set_option('display.max_columns', None)
    pd.set_option('display.max_rows', None)
    
def nf():
    pd.set_option('display.max_columns', None)
    pd.set_option('display.max_rows', 4)
nf()
import warnings
warnings.filterwarnings('ignore')

def dp(df, r = 5, c = None):
    from IPython.display import display
    with pd.option_context('display.max_rows', 4, 'display.max_columns', None):
        display(df)

def fg(w = 6, h = 4, dpi = 120):
    plt.rcParams['figure.figsize'] = (w,h)
    plt.rcParams['figure.dpi'] = dpi
fg()

# import autotime
# %load_ext autoreload
# %autoreload 2
# %load_ext autotime

# For dark themes 
from jupyterthemes import jtplot
jtplot.style()

# Para quando tiver atualização de submodulos
%load_ext autoreload
%autoreload 2

In [2]:
df = pd.read_csv("data/UCI_Credit_Card.csv")

In [3]:
df.shape

(30000, 25)

In [4]:
df.columns = df.columns.str.lower()

In [5]:
df

,id,limit_bal,sex,education,marriage,age,pay_0,pay_2,pay_3,pay_4,pay_5,pay_6,bill_amt1,bill_amt2,bill_amt3,bill_amt4,bill_amt5,bill_amt6,pay_amt1,pay_amt2,pay_amt3,pay_amt4,pay_amt5,pay_amt6,default.payment.next.month
0,1,20000.0,2,2,1,24,2,2,-1,-1,-2,-2,3913.0,3102.0,689.0,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1
1,2,120000.0,2,2,2,26,-1,2,0,0,0,2,2682.0,1725.0,2682.0,3272.0,3455.0,3261.0,0.0,1000.0,1000.0,1000.0,0.0,2000.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29998,29999,80000.0,1,3,1,41,1,-1,0,0,0,-1,-1645.0,78379.0,76304.0,52774.0,11855.0,48944.0,85900.0,3409.0,1178.0,1926.0,52964.0,1804.0,1
29999,30000,50000.0,1,2,1,46,0,0,0,0,0,0,47929.0,48905.0,49764.0,36535.0,32428.0,15313.0,2078.0,1800.0,1430.0,1000.0,1000.0,1000.0,1


In [6]:
df.pay_5.unique()

array([-2,  0, -1,  2,  3,  5,  4,  7,  8,  6])

In [7]:
f()
df.dtypes
nf()

id                              int64
limit_bal                     float64
sex                             int64
education                       int64
marriage                        int64
age                             int64
pay_0                           int64
pay_2                           int64
pay_3                           int64
pay_4                           int64
pay_5                           int64
pay_6                           int64
bill_amt1                     float64
bill_amt2                     float64
bill_amt3                     float64
bill_amt4                     float64
bill_amt5                     float64
bill_amt6                     float64
pay_amt1                      float64
pay_amt2                      float64
pay_amt3                      float64
pay_amt4                      float64
pay_amt5                      float64
pay_amt6                      float64
default.payment.next.month      int64
dtype: object

In [8]:
df = df.drop('id', axis = 1)

In [9]:
df = df.rename(columns = {'default.payment.next.month':'target'})

In [10]:
! ls

_01_dataprep.ipynb					  feature_selector
_02_model_with_average_precision_feature_selection.ipynb  libs
_03_lightgbm_model_tuning_parameters.ipynb		  lightbmparameters.txt
data


In [11]:
df.to_parquet('data/df_to_feature_selection.parquet', index = False)

In [12]:
df.target.value_counts()

0    23364
1     6636
Name: target, dtype: int64

# LGBM

In [13]:
import lightgbm as lgb

In [14]:
clf = lgb.LGBMClassifier()

In [15]:
from libs.pipeline import Pipe

In [16]:
df

,limit_bal,sex,education,marriage,age,pay_0,pay_2,pay_3,pay_4,pay_5,pay_6,bill_amt1,bill_amt2,bill_amt3,bill_amt4,bill_amt5,bill_amt6,pay_amt1,pay_amt2,pay_amt3,pay_amt4,pay_amt5,pay_amt6,target
0,20000.0,2,2,1,24,2,2,-1,-1,-2,-2,3913.0,3102.0,689.0,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1
1,120000.0,2,2,2,26,-1,2,0,0,0,2,2682.0,1725.0,2682.0,3272.0,3455.0,3261.0,0.0,1000.0,1000.0,1000.0,0.0,2000.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29998,80000.0,1,3,1,41,1,-1,0,0,0,-1,-1645.0,78379.0,76304.0,52774.0,11855.0,48944.0,85900.0,3409.0,1178.0,1926.0,52964.0,1804.0,1
29999,50000.0,1,2,1,46,0,0,0,0,0,0,47929.0,48905.0,49764.0,36535.0,32428.0,15313.0,2078.0,1800.0,1430.0,1000.0,1000.0,1000.0,1


In [36]:
ma = Pipe(df = df, model = clf, target_var = 'target')

[[0.68568533 0.31431467]
 [0.93250015 0.06749985]
 [0.36994943 0.63005057]
 ...
 [0.85846754 0.14153246]
 [0.76407872 0.23592128]
 [0.73892085 0.26107915]]


In [31]:
ma.y_train

28465    1
27622    0
        ..
15795    1
23654    1
Name: target, Length: 21000, dtype: int64

In [32]:
ma.y_test

2308     0
22404    0
        ..
6279     0
26428    0
Name: target, Length: 9000, dtype: int64

In [ ]:
ma.

In [27]:
ma.popin.scores()

ValueError: Classification metrics can't handle a mix of binary and multilabel-indicator targets